In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import tensorflow as tf
import numpy as np
import cv2 as cv

In [ ]:
# Defining meta-information for model
IMG_WIDTH = 512
IMG_HEIGHT = 512
interpolation_method = cv.INTER_LANCZOS4 # Reasoning: also lanczos here

# Defining Paths
dirname = os.path.dirname(__file__)
base_path = os.path.join(dirname, 'correct_path_to_dataset/4_cleaning/4_cleaning/') # define the correct path
train_path = base_path + 'Train/'
test_path = base_path + 'Test/'

train_ids_dir = os.path.join(train_path, 'Ids')
train_masks_dir = os.path.join(train_path, 'GroundTruth')

train_ids_list = sorted(os.listdir(train_ids_dir))
train_masks_list = sorted(os.listdir(train_masks_dir))

test_ids_dir = os.path.join(test_path, 'Ids')
test_masks_dir = os.path.join(test_path, 'GroundTruth')

test_ids_list = sorted(os.listdir(test_ids_dir))
test_masks_list = sorted(os.listdir(test_masks_dir))


# Handle Training Data
train_images = []
train_masks = []

for img_name, mask_name in zip(train_ids_list, train_masks_list):
    img_path = os.path.join(train_ids_dir, img_name)
    mask_path = os.path.join(train_masks_dir, mask_name)

    # Load and preprocess images
    img = cv.imread(img_path)
    img = cv.resize(img,(IMG_WIDTH,IMG_HEIGHT),interpolation=interpolation_method)
    train_images.append(img)

    # Load and preprocess masks (assuming they are grayscale)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    mask = cv.resize(mask,(IMG_WIDTH,IMG_HEIGHT),interpolation=interpolation_method)
    train_masks.append(mask)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_masks = np.array(train_masks)

# Normalize pixel values if needed (e.g., scale to [0, 1]) (done in next code cell)
train_images = train_images / 255.0
train_masks = train_masks / 255.0


# Handle Testing Data
test_images = []
test_masks = []

for img_name, mask_name in zip(test_ids_list, test_masks_list):
    img_path = os.path.join(test_ids_dir, img_name)
    mask_path = os.path.join(test_masks_dir, mask_name)

    # Load and preprocess images
    img = cv.imread(img_path)
    img = cv.resize(img,(IMG_WIDTH,IMG_HEIGHT),interpolation=interpolation_method)
    test_images.append(img)

    # Load and preprocess masks (assuming they are grayscale)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    mask = cv.resize(mask,(IMG_WIDTH,IMG_HEIGHT),interpolation=interpolation_method)
    test_masks.append(mask)

# Convert lists to numpy arrays
test_images = np.array(test_images)
test_masks = np.array(test_masks)

# Normalize pixel values if needed (e.g., scale to [0, 1]) (done in next code cell)
test_images = test_images / 255.0
test_masks = test_masks / 255.0

In [ ]:
# Defining our U-Net
IMG_CHANNELS = 3

inputs = tf.keras.layers.Input((IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))

# downsampling
c1 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.3)(c4)
c4 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

# upsampling
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2))(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2))(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2))(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2))(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
# https://stackoverflow.com/questions/63952338/how-to-save-best-weights-and-best-model-using-keras
checkpoint = None
callbacks_list = None
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/ipda/models/ipda_task_4_clean_best_weights.h5', monitor='val_binary_io_u', verbose=1, save_best_only=True,  mode='max')
callbacks_list = [checkpoint]

# Why adam for optimizer? Generally good performance according to https://stackoverflow.com/questions/37214884/how-do-i-choose-an-optimizer-for-my-tensorflow-model and common use in practice according to lecture slides
# Why binary_focal_crossentropy for loss? We work with binary segmentation (imbalanced problem).
# Why accuracy for metrics? Imbalanced problem, so we add Recall to the mix insteadof just accuracy
model.compile(optimizer='adam', loss='binary_focal_crossentropy', metrics=['accuracy','Recall',tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=0.5)])
fitting_information = model.fit(train_images, train_masks, validation_split=0.2, epochs=100, batch_size=8,callbacks=callbacks_list)

In [ ]:
# Test model and print validation and test accuracies
model.load_weights('/content/drive/My Drive/ipda/models/ipda_task_4_clean_best_weights.h5')
test_loss, test_acc, test_recall, test_binary_io_u = model.evaluate(test_images, test_masks)

print(f"training loss: {fitting_information.history['loss'][-1]}, training accuracy: {fitting_information.history['accuracy'][-1]}, training recall: {fitting_information.history['recall'][-1]}, training binary_io_u: {fitting_information.history['binary_io_u'][-1]}")
print(f"validation loss: {fitting_information.history['val_loss'][-1]}, validation accuracy: {fitting_information.history['val_accuracy'][-1]}, validation recall: {fitting_information.history['val_recall'][-1]}, validation binary_io_u: {fitting_information.history['val_binary_io_u'][-1]}")
print(f"test loss: {test_loss}, test accuracy: {test_acc}, test recall: {test_recall}, test binary_io_u: {test_binary_io_u}")

In [ ]:
# Save Model
model.save('/content/drive/My Drive/ipda/models/ipda_task_4_clean_2')
model_save_path = os.path.join(dirname,'models/ipda_task_4_clean_3')

# Testing

In [ ]:
print(f"type of test_images[0]: {type(test_images[0])}")
print(f"shape of test_images[0]: {test_images[0].shape}")
img_task_4_resized = np.expand_dims(test_images[0],axis=0)
prediction = model.predict(img_task_4_resized)
print(f"type of prediction: {type(prediction)}")
print(f"shape of prediction: {prediction.shape}")

import matplotlib.pyplot as plt

plt.figure(figsize=(60, 30))

plt.subplot(1, 2, 1)
plt.imshow(test_images[0])
plt.title('Image Resized for Task 1')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(prediction[0],cmap='gray')
plt.title('Predicted Mask Task 2')
plt.axis('off')